# Convex hull as an index to support an algorithm for the mutualisation of itineraires - to see it renderd visit https://nbviewer.org/github/Universite-Gustave-Eiffel/geodev-logicout/blob/Thiago/notebooks/convex_hull.ipynb

In [2]:
import sys  
sys.path.insert(0, '../scripts')
import use_data
import IsInclude
from shapely import wkt
import geopandas as gpd

In [3]:
# we initialize the parameters for the geodataframe
dist = 100000
filename = "simulations_reel_gdf.csv"
gdf = use_data.create_gdf(filename,'cheflieu')

In [4]:
dist = 100000

sampleA = gdf.iloc[[1]]
id_simu_A = sampleA['id_simulation']

id_simu_tot = IsInclude.IsIn_tournee_gdf(sampleA, gdf, dist, 1)['id_simulation_right'].values
id_simu_double = IsInclude.IsIn_tournee_gdf(sampleA, gdf, dist, 2)['id_simulation_right'].values
id_simu_simple = []
for id in id_simu_tot:
    if id not in id_simu_double:
        id_simu_simple.append(id)

id_simu_B = id_simu_simple[8]
id_simu_C = id_simu_double[0]

id_B = gdf.index[gdf['id_simulation']==id_simu_B][0]
id_C = gdf.index[gdf['id_simulation']==id_simu_C][0]

sampleB = gdf.iloc[[id_B]] # simple inclusion
sampleC = gdf.iloc[[id_C]] # double inclusion

In [5]:
#this function will return a map and a geodataframe consisting of all mutualisables itineraires for the given sample
def exemple_inclusion(sampleA, sampleB, color):
    # Affichage du point de départ de la tournée A
    sampleA = gpd.GeoDataFrame(sampleA,geometry=sampleA['start'].map(wkt.loads))
    m = sampleA.explore(tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=4,color='black'))
    # Affichage de l'itinéraire de la tournée A
    sampleA = gpd.GeoDataFrame(sampleA,geometry=sampleA['itineraire'].map(wkt.loads))
    m = sampleA.explore(m=m,tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=2,color='black'))
    # Affichage du cercle de rayon dist et de centre le chef-lieu de la commune de la tournée A
    geometryA = sampleA['cheflieu'].map(wkt.loads)
    sampleA = gpd.GeoDataFrame(sampleA, geometry=geometryA, crs = 'EPSG:2154') # we get the center of the cheflieu as a geometry
    sampleA['buffer'] = sampleA.geometry.buffer(dist) # create the buffer based on the cheflieu point
    sampleA = gpd.GeoDataFrame(sampleA, geometry='buffer')
    m = sampleA.explore(m=m,tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=2,color='black'))

    # Affichage du point de départ de la tournée B
    sampleB = gpd.GeoDataFrame(sampleB,geometry=sampleB['start'].map(wkt.loads))
    m = sampleB.explore(m=m, tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=4,color=color))
    # Affichage de l'itinéraire de la tournée A
    sampleB = gpd.GeoDataFrame(sampleB,geometry=sampleB['itineraire'].map(wkt.loads))
    m = sampleB.explore(m=m,tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=2,color=color))
    # Affichage du cercle de rayon dist et de centre le chef-lieu de la commune de la tournée A
    geometryB = sampleB['cheflieu'].map(wkt.loads)
    sampleB = gpd.GeoDataFrame(sampleB, geometry=geometryB, crs = 'EPSG:2154') # we get the center of the cheflieu as a geometry
    sampleB['buffer'] = sampleB.geometry.buffer(dist) # create the buffer based on the cheflieu point
    sampleB = gpd.GeoDataFrame(sampleB, geometry='buffer')
    m = sampleB.explore(m=m,tiles='CartoDB positron',style_kwds=dict(fill=False, stroke=True,weight=2,color=color))

    return m

sample_map_simple = exemple_inclusion(sampleA, sampleB, 'red')
sample_map_double = exemple_inclusion(sampleA, sampleC, 'green')


In [6]:
# in black the itineraire of the chosen sample
sample_map_simple

In [7]:
output= "pages_html/simple_inclusion.html"
sample_map_simple.save(output)

In [8]:
sample_map_double

In [9]:
output= "pages_html/double_inclusion.html"
sample_map_double.save(output)

In [10]:
gdf_to_explore = use_data.create_gdf(filename,'start')


In [14]:
gdf_to_explore.explore(tiles='CartoDB positron',color='red',style_kwds=dict(fill=True,fillOpacity=0.1,stroke=False,radius=5))

In [16]:
gdf_id_unique = gdf_to_explore['id_utilisateur'].nunique()
gdf_id_unique

150

In [15]:
gdf_start_unique = gdf_to_explore['start'].nunique()
gdf_start_unique

227